In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from train_transformer import get_tokenizer, get_model, freeze_layers, create_evaluate, get_metric, train_transformer, prepare_dataset, test_model


In [24]:
from datasets import load_dataset
from functools import partial
LIMIT=1000
tokenizer = get_tokenizer("ufal/robeczech-base")
prepare_partial =  partial(prepare_dataset, tokenizer, 512, 4, "server", LIMIT)


In [25]:
train_dataset = prepare_partial(load_dataset("../..//final_dataset/dataset.py", split="train"))
test_dataset = prepare_partial(load_dataset("../../final_dataset/dataset.py", split="test"))
eval_dataset = prepare_partial(load_dataset("../../final_dataset/dataset.py", split="validation"))

Found cached dataset dataset (/home/kydliceh/.cache/huggingface/datasets/dataset/default/0.0.0/dd40f907db796b1c96c399ea8c9ec6f99ebe75363e9a0f0f63f6a15c6f03423f)


The OrderedVocab you are attempting to save contains a hole for index 51959, your vocabulary could be corrupted !


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Found cached dataset dataset (/home/kydliceh/.cache/huggingface/datasets/dataset/default/0.0.0/dd40f907db796b1c96c399ea8c9ec6f99ebe75363e9a0f0f63f6a15c6f03423f)


...

Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Found cached dataset dataset (/home/kydliceh/.cache/huggingface/datasets/dataset/default/0.0.0/dd40f907db796b1c96c399ea8c9ec6f99ebe75363e9a0f0f63f6a15c6f03423f)


The OrderedVocab you are attempting to save contains a hole for index 51959, your vocabulary could be corrupted !


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [28]:
out_dim = train_dataset.features["label"].num_classes - 1

In [29]:
from train_transformer import get_model, freeze_layers, get_metric, create_evaluate

score_type = ["accuracy", "f1-macro"]

model = get_model("ufal/robeczech-base", out_dim)
model = freeze_layers(model, 2)
metrics = {metric: get_metric(metric) for metric in score_type}
evaluator = create_evaluate(metrics)


Some weights of the model checkpoint at ufal/robeczech-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ufal/robeczech-base and are newly initialized: 

In [30]:
from pathlib import Path
from datetime import datetime
output = Path("models") / datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [33]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [34]:
print_gpu_utilization()

<function __main__.print_gpu_utilization()>

In [ ]:
from train_transformer import train_transformer
from pathlib import Path
model = train_transformer(model, tokenizer, train_dataset, eval_dataset, Path("models"), evaluator, logging_steps=1, eval_steps=1)

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: category, date, authors_gender, url, headline, content, authors_cum_gender, authors, day_of_week, brief. If category, date, authors_gender, url, headline, content, authors_cum_gender, authors, day_of_week, brief are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/lnet/aic/personal/kydliceh/non_runable/NLP_venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 2
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


: 

: 